In [1]:
!pip install requests beautifulsoup4 pandas matplotlib -q

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import Dict, List, Optional
import re
from dataclasses import dataclass
from urllib.parse import urljoin
import time

## Подготовка данных для векторной базы данных

### HTML - Страница сайтов образовательных программ
Используем библиотеки requests - чтобы загрузить необходимые старницы, BeautifulSoup - для изалечения текста из html-страницы

In [3]:
URLS = {
    "ai": "https://abit.itmo.ru/program/master/ai",
    "ai_product": "https://abit.itmo.ru/program/master/ai_product"
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def fetch_page(url: str) -> Optional[str]:
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке {url}: {e}")
        return None

In [4]:
pages = {}
for name, url in URLS.items():
    html_content = fetch_page(url)
    if html_content:
        pages[name] = html_content
        print(f"Страница '{name}' успешно загружена ({len(html_content)} символов)")
    else:
        print(f"Не удалось загрузить страницу '{name}'")

Страница 'ai' успешно загружена (197311 символов)
Страница 'ai_product' успешно загружена (137569 символов)


In [5]:
ai_page = pages["ai"]
soup = BeautifulSoup(ai_page, 'html.parser')
for ai_data in soup(['style', 'script']):
  ai_data.decompose()
ai_text = ' '.join(soup.stripped_strings)

In [6]:
ai_text[:1000]

'Искусственный интеллект itmo.ru +7 (812) 480-0-480 abit@itmo.ru Eng Бакалавриат Магистратура Аспирантура Дополнительное образование ЛИЧНЫЙ КАБИНЕТ Бакалавриат Магистратура Основная информация Дополнительное образование Аспирантура Личный кабинет Образовательные программы магистратуры подать документы Искусственный интеллект институт прикладных компьютерных наук форма обучения очная длительность 2 года язык обучения русский общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности СОП, Онлайн, Трек аспирантуры, ПИШ, Программа в сфере ИИ Менеджер программы Елизавета Витальевна Василенко aitalents@itmo.ru +7 (999) 526-79-88 Программа в соцсетях ВКонтакте Сайт Telegram направления подготовки 27.04.05 Инноватика 80 бюджетных 0 целевая 20 контрактных 09.04.01 Информатика и вычислительная техника 50 бюджетных 0 целевая 50 контрактных 11.04.02 Инфокоммуникационные технологии и системы связи 85 бюджетных 0 целевая 25 контрактных о программе Создавайте AI-продукты и 

In [7]:
ai_product_page = pages["ai_product"]
soup = BeautifulSoup(ai_product_page, 'html.parser')
for ai_product_data in soup(['style', 'script']):
  ai_product_data.decompose()
ai_product_text = ' '.join(soup.stripped_strings)

In [8]:
ai_product_text[:1000]

'Управление ИИ-продуктами / AI Product itmo.ru +7 (812) 480-0-480 abit@itmo.ru Eng Бакалавриат Магистратура Аспирантура Дополнительное образование ЛИЧНЫЙ КАБИНЕТ Бакалавриат Магистратура Основная информация Дополнительное образование Аспирантура Личный кабинет Образовательные программы магистратуры подать документы Управление ИИ-продуктами / AI Product институт прикладных компьютерных наук форма обучения очная длительность 2 года язык обучения русский общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности СОП, Программа в сфере ИИ Менеджер программы Регина Ильдаровна Абдрашитова aiproduct@itmo.ru +7 (993) 639-86-77 Программа в соцсетях ВКонтакте Сайт Telegram направления подготовки 02.04.03 Математическое обеспечение и администрирование информационных систем 26 бюджетных 0 целевая 25 контрактных о программе Создавайте AI-продукты и технологии, которые меняют мир. Программа дает глубокие технические знания в области разработки систем искусственного интелле

### PDF - Учебные планы
Используем библиотеку pdfplumber для работы с pdf-файлами

In [9]:
!pip install pdfplumber -q

In [22]:
import pandas as pd
import json
import re
from typing import Dict, List, Any
import os

import requests

def download_pdf(url, filename):
    try:
        if 'drive.google.com' in url:
            file_id = url.split('/d/')[1].split('/')[0]
            direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'
        else:
            direct_url = url
        
        response = requests.get(direct_url)
        response.raise_for_status()
        
        with open(filename, 'wb') as f:
            f.write(response.content)
        
        print(f"✓ PDF файл скачан: {filename} ({len(response.content)} байт)")
        return True
    except Exception as e:
        print(f"✗ Ошибка загрузки PDF: {e}")
        return False

ai_pdf_url = "https://drive.google.com/file/d/1NSuBA0VkPzqNfdOD2IDR4TSDY2ea66nx/view"
ai_pdf_filename = "УП_Искусственный интеллект.pdf"
ai_product_pdf_url = "https://drive.google.com/file/d/1Uod9BdkrFqSfiCq6w2nLHQC-tPW_NNRJ/view"
ai_product_pdf_filename = "Учебный план AI Product.pdf"

download_pdf(ai_pdf_url, ai_pdf_filename)
download_pdf(ai_product_pdf_url, ai_product_pdf_filename)

✓ PDF файл скачан: УП_Искусственный интеллект.pdf (88123 байт)
✓ PDF файл скачан: Учебный план AI Product.pdf (79489 байт)


True

In [23]:
import pdfplumber
import pandas as pd
import json
import re
import os
from typing import Dict, List, Any

def extract_text_with_pdfplumber(pdf_path: str) -> Dict:    
    if not os.path.exists(pdf_path):
        print(f"Файл {pdf_path} не найден")
        return {}
    
    try:
        full_text = ""
        page_texts = []
        
        with pdfplumber.open(pdf_path) as pdf:
            print(f"✓ PDF открыт: {len(pdf.pages)} страниц")
            
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                page_texts.append({
                    "page": page_num + 1,
                    "text": text
                })
                full_text += f"\n--- Страница {page_num + 1} ---\n{text}"
        
        return {
            "total_pages": len(pdf.pages),
            "full_text": full_text,
            "pages": page_texts,
            "text_length": len(full_text)
        }
    
    except Exception as e:
        print(f"Ошибка чтения PDF: {e}")
        return {}

def show_text_pdf_shortly(pdf_data):
    if pdf_data:
        print(f"Страниц: {pdf_data['total_pages']}")
        print(f"Символов: {pdf_data['text_length']}")
        
        print("\nПервые 1000 символов:")
        print("-" * 50)
        print(pdf_data['full_text'][:1000])
        print("-" * 50)

ai_pdf_data = extract_text_with_pdfplumber(ai_pdf_filename)
show_text_pdf_shortly(ai_pdf_data)
print('\n\n\n')
ai_product_pdf_data = extract_text_with_pdfplumber(ai_product_pdf_filename)
show_text_pdf_shortly(ai_product_pdf_data)

✓ PDF открыт: 5 страниц
Страниц: 5
Символов: 11760

Первые 1000 символов:
--------------------------------------------------

--- Страница 1 ---
Учебный план
ОП Искусственный интеллект
атратс
ыртсемеС Наименование модулей, дисциплин, практики и аттестации
де.з
в
ьтсокмеодурТ
.сач
в
ьтсокмеодурТ
Блок 1. Модули (дисциплины) 60 2160
Универсальная (надпрофессиональная) подготовка 12 432
Магистратура/Аспирантура ИИ 9 324
Мировоззренческий модуль + иняз 9 324
Иностранный язык 6 216
Иностранный язык 2 сем 3 108
2 Английский язык A1 / English A1 3 108
2 Английский язык A2 / English A2 3 108
2 Русский язык как иностранный / Russian as a foreign language 3 108
2 Английский язык в профессиональной деятельности / English for specific purposes 3 108
Иностранный язык 1 сем 3 108
1 Английский язык A1 / English A1 3 108
1 Английский язык A2 / English A2 3 108
1 Русский язык как иностранный / Russian as a foreign language 3 108
1 Английский язык в профессиональной деятельности / English for specific pu

### Datapipeline

In [37]:
%%capture

!pip install spacy
!python -m spacy download ru_core_news_md
!pip install tdqm

In [ ]:
import spacy
from tqdm.auto import tqdm

nlp = spacy.load("ru_core_news_md")
nlp.disable_pipes(['ner', 'parser'])

def process_text(text, show_token_progress=True):
    doc = nlp(text)    
    total_tokens = len(doc)
    
    if show_token_progress:
        # Создаем прогресс-бар для обработки токенов
        tokens = []
        for token in tqdm(doc, total=total_tokens, desc="Обработка токенов", 
                          unit="токен"):
            if (not token.is_stop and 
                not token.is_punct and 
                not token.is_digit and
                not token.like_email and
                not token.like_num and
                not token.is_space):
                tokens.append(token.lemma_.lower())
    else:
        # Обычная обработка без прогресс-бара на уровне токенов
        tokens = [
            token.lemma_.lower() for token in doc
            if not token.is_stop
            and not token.is_punct
            and not token.is_digit
            and not token.like_email
            and not token.like_num
            and not token.is_space
        ]
    
    return ' '.join(tokens)

In [14]:
ai_text_preprocess = process_text(ai_text)

Обработка токенов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2026/2026 [00:00<00:00, 1087630.86токен/s]


In [15]:
ai_text_preprocess[:1000]

'искусственный интеллект itmo.ru eng бакалавриат магистратура аспирантура дополнительный образование личный кабинет бакалавриат магистратура основный информация дополнительный образование аспирантура личный кабинет образовательный программа магистратура подать документ искусственный интеллект институт прикладной компьютерный наука форма обучение очный длительность год язык обучение русский общежитие военный учебный центр гос аккредитация дополнительный возможность соп онлайн трек аспирантура пиш программа сфера ии менеджер программа елизавета виталиевич василенко программа соцсетях вконтакте сайт telegram направление подготовка инноватика бюджетный целевой контрактный информатика вычислительный техника бюджетный целевой контрактный инфокоммуникационные технология система связь бюджетный целевой контрактный программа создавать ai продукт технология менять мир основа обучение программа проектный подход магистрант работать проект ведущий компания x5 group ozon банк мтс sber ai норникель n

In [16]:
ai_product_text_preprocess = process_text(ai_product_text)
ai_product_text_preprocess[:1000]

Обработка токенов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1457/1457 [00:00<00:00, 1160702.93токен/s]


'управление ии продукт ai product itmo.ru eng бакалавриат магистратура аспирантура дополнительный образование личный кабинет бакалавриат магистратура основный информация дополнительный образование аспирантура личный кабинет образовательный программа магистратура подать документ управление ии продукт ai product институт прикладной компьютерный наука форма обучение очный длительность год язык обучение русский общежитие военный учебный центр гос аккредитация дополнительный возможность соп программа сфера ии менеджер программа регин ильдарович абдрашитов программа соцсетях вконтакте сайт telegram направление подготовка математический обеспечение администрирование информационный система бюджетный целевой контрактный программа создавать ai продукт технология менять мир программа давать глубокий технический знание область разработка система искусственный интеллект навык продуктовый менеджмент смочь создавать инновационный ии‑решения выводить рынок широкий выбор предмет позволять построить инд

Удаляем муссорный блок текста в обоих pdf файлах

In [27]:
import re

pattern = r'^атратс\s*[\s\S]*?(?=Блок 1\.)'

In [24]:
ai_study_plan = ai_pdf_data['full_text']

In [26]:
ai_study_plan = re.sub(pattern, '', ai_study_plan, flags=re.MULTILINE)
print(ai_study_plan[:1000])


--- Страница 1 ---
Учебный план
ОП Искусственный интеллект
Блок 1. Модули (дисциплины) 60 2160
Универсальная (надпрофессиональная) подготовка 12 432
Магистратура/Аспирантура ИИ 9 324
Мировоззренческий модуль + иняз 9 324
Иностранный язык 6 216
Иностранный язык 2 сем 3 108
2 Английский язык A1 / English A1 3 108
2 Английский язык A2 / English A2 3 108
2 Русский язык как иностранный / Russian as a foreign language 3 108
2 Английский язык в профессиональной деятельности / English for specific purposes 3 108
Иностранный язык 1 сем 3 108
1 Английский язык A1 / English A1 3 108
1 Английский язык A2 / English A2 3 108
1 Русский язык как иностранный / Russian as a foreign language 3 108
1 Английский язык в профессиональной деятельности / English for specific purposes 3 108
Мировоззренческий модуль 3 108
Предпринимательская культура 3 108
3 Создание и развитие технологического бизнеса 3 108
3 Стартап-трек: от mvp до бизнеса 3 108
Креативные технологии 3 108
1 Основы концептуального мышления 3 

In [28]:
ai_product_study_plan = ai_product_pdf_data['full_text']
ai_product_study_plan = re.sub(pattern, '', ai_product_study_plan, flags=re.MULTILINE)
print(ai_product_study_plan[:1000])


--- Страница 1 ---
Учебный план
ОП Управление ИИ-продуктами/AI Product
Блок 1. Модули (дисциплины) 72 2592
Индивидуальная профессиональная подготовка (по профессиональным областям, по
60 2160
профессиональным ролям, по уровню сложности и др.)
Обязательные дисциплины. 1 семестр 6 216
1 Продуктовые исследования 3 108
1 Воркшоп по созданию продукта на данных / Data Product Development Workshop 3 108
Пул выборных дисциплин. 1 семестр 12 432
1 Процессы и методологии разработки решений на основе ИИ 3 108
1 Монетизация ИИ-продуктов 3 108
1 Стратегический продуктовый менеджмент 3 108
1 Продуктовый дизайн и прототипирование AI-решений 3 108
1 Математика для машинного обучения и анализа данных 3 108
1 Математическая статистика 3 108
1 Основы программирования на Python 3 108
1 Основы машинного обучения 3 108
1 Основы глубокого обучения 3 108
1 Введение в большие языковые модели (LLM) 3 108
1 Прикладной анализ временных рядов 3 108
1 Инженерные практики в ML и анализе данных 6 216
1 Прикладные ин

Применяем предобработку для учебных планов

In [29]:
ai_study_plan_preprocess = process_text(ai_study_plan)

Обработка токенов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2348/2348 [00:00<00:00, 1393156.85токен/s]


In [30]:
ai_study_plan_preprocess[:1000]

'страница учебный план искусственный интеллект блок модуль дисциплина универсальный надпрофессиональная подготовка магистратура аспирантура ии мировоззренческий модуль + иняз иностранный язык иностранный язык сема английский язык a1 english a1 английский язык a2 english a2 русский язык иностранный russian as a foreign language английский язык профессиональный деятельность english for specific purposes иностранный язык сема английский язык a1 english a1 английский язык a2 english a2 русский язык иностранный russian as a foreign language английский язык профессиональный деятельность english for specific purposes мировоззренческий модуль предпринимательский культура создание развитие технологический бизнес стартап трек mvp бизнес креативный технология основа концептуальный мышление основа концептуальный мышление introduction to conceptual thinking мышление этика сфера информационный технология искусственный интеллект critical thinking skills advanced критический мышление продвинутый урове

In [31]:
ai_product_study_plan_preprocess = process_text(ai_product_study_plan)
ai_product_study_plan_preprocess[:1000]

Обработка токенов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1435/1435 [00:00<00:00, 1417528.55токен/s]


'страница учебный план управление ии продукт ai product блок модуль дисциплина индивидуальный профессиональный подготовка профессиональный область профессиональный роль уровень сложность и др. обязательный дисциплина семестр продуктовый исследование воркшоп создание продукт data product development workshop пул выборный дисциплина семестр процессы методология разработка решение основа ии монетизация ии продукт стратегический продуктовый менеджмент продуктовый дизайн прототипирование ai решение математик машинный обучение анализ математический статистика основа программирование python основа машинный обучение основа глубокий обучение введение больший языковой модель llm прикладной анализ временной ряд инженерный практика ml анализ прикладной инструмент разработка разработка веб приложений python backend проектирование микросервисов пул выборный дисциплина семестр бизнес анализ практика менторства развитие data science управление проект data science метрика аналитик продукт управление пр

### Сохранение для данных для векторной базы

In [38]:
import os
os.mkdir('../data')

In [40]:
with open("../data/ai_program_info.txt", "w", encoding="utf-8") as f:
    f.write(ai_text_preprocess)

with open("../data/ai_product_program_info.txt", "w", encoding="utf-8") as f:
    f.write(ai_product_text_preprocess)

with open("../data/ai_study_plan.txt", "w", encoding="utf-8") as f:
    f.write(ai_study_plan_preprocess)
    
with open("../data/ai_product_study_plan.txt", "w", encoding="utf-8") as f:
    f.write(ai_product_study_plan_preprocess)

## Настройка LLM

In [60]:
OPENAI_API_KEY = ""

In [43]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [44]:
%%capture

!pip install llama-index


In [46]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os

documents = SimpleDirectoryReader("../data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

2026-02-03 08:42:48,478 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [47]:
from llama_index.core import Settings

async def search_documents(query: str) -> str:
    response = await query_engine.aquery(query)
    return str(response)

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

agent = FunctionAgent(
    tools=[search_documents],
    llm=Settings.llm,
    system_prompt="""Ты ассистент, который помогает абитуриенту разобарться, какая из двух магистерских программ - 
    Искусственный интеллект и Управление ИИ-продуктами / AI Product,
    подходит ему лучше. В твоей базе есть содержимое html-страниц этих программ и учебные планы. 
    Умей такжее отвечать на вопросы по содержимому твоей базы. Твои ответы будут в телеграм боте, не используй markdown""",
)

### Тестирование

In [48]:
response = await agent.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-03 08:43:09,331 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:09,988 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:43:10,838 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:43:12,167 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:13,561 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:15,270 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У меня есть две магистерские программы:

1. **Искусственный интеллект**: Эта программа охватывает различные области, такие как наука о данных, машинное обучение и глубокое обучение. Учебный план включает темы, такие как программирование на Python, разработка продуктов на основе данных, продвинутое машинное обучение и обработка естественного языка. Студенты имеют возможность работать над реальными проектами с ведущими компаниями в этой области, получая практический опыт и возможности для налаживания контактов.

2. **Управление ИИ-продуктами**: Эта программа сосредоточена на управлении продуктами на основе искусственного интеллекта и предоставляет глубокие технические знания в разработке систем ИИ, а также навыки в управлении продуктами. Выпускники могут работать в качестве менеджеров продуктов ИИ, менеджеров проектов ИИ, аналитиков данных продуктов и других специалистов. Учебный план включает модули по исследованию продуктов, разработке, стратегическому управлению продуктами, монетизаци

In [49]:
response = await agent.run(
        "Хочу уметь создавать микросервисы. Подскажи на какую программу пойти лучше"
    )
print(response)

2026-02-03 08:43:25,021 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:25,403 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:43:25,420 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:43:27,780 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:28,430 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:29,119 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Если вы хотите уметь создавать микросервисы, вам стоит обратить внимание на программу "Искусственный интеллект". Она предлагает курсы по машинному обучению, анализу данных и разработке ИИ-продуктов. Студенты имеют возможность работать над проектами с ведущими компаниями и могут стать инженерами по машинному обучению, инженерами данных или разработчиками ИИ-продуктов.

Программа "Управление ИИ-продуктами" больше ориентирована на управление продуктами и включает в себя навыки продуктового менеджмента и создание инновационных ИИ-решений. Хотя она также может быть полезной, если вас интересует именно разработка и создание микросервисов, первая программа будет более подходящей.


In [50]:
response = await agent.run(
        "Хочу уметь создавать микросервисы. Подскажи на какую программу пойти лучше изходя из учебного плана"
    )
print(response)

2026-02-03 08:43:49,181 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:49,751 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:43:49,768 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:43:51,097 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:52,197 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:43:54,474 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Если вы хотите уметь создавать микросервисы, программа "Управление ИИ-продуктами" может быть более подходящей для вас. В её учебном плане есть дисциплины, связанные с проектированием и разработкой микросервисов, а также веб-приложений. 

Программа "Искусственный интеллект" охватывает более широкий спектр тем, связанных с искусственным интеллектом, но не акцентирует внимание на разработке микросервисов. 

Таким образом, если ваша цель - создание микросервисов, выбирайте программу "Управление ИИ-продуктами".


In [51]:
response = await agent.run(
        "Я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере"
    )
print(response)

2026-02-03 08:44:12,974 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Учитывая ваш опыт в Java и Python, а также интерес к созданию ML-сервисов, обе магистерские программы могут быть вам интересны. 

Программа "Искусственный интеллект" может углубить ваши знания в области алгоритмов, машинного обучения и нейронных сетей, что будет полезно для разработки более сложных ML-сервисов.

Программа "Управление ИИ-продуктами" может быть более ориентирована на аспекты управления проектами, продуктами и командами в сфере ИИ, что может быть полезно, если вы хотите развиваться в направлении управления проектами или стартапами.

Если вы хотите углубить технические навыки и знания в области ИИ, выбирайте программу "Искусственный интеллект". Если вас больше интересует управление продуктами и проектами в этой области, то "Управление ИИ-продуктами" будет более подходящей.

Если вам нужны подробности о каждой программе, дайте знать!


In [52]:
response = await agent.run(
        "Подскажи какую программу, мне лучше выбрать"
    )
print(response)

2026-02-03 08:44:17,856 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Чтобы помочь вам выбрать между программами "Искусственный интеллект" и "Управление ИИ-продуктами", мне нужно больше информации о ваших интересах и целях. 

1. Какой у вас опыт в области ИИ и технологий?
2. Какие навыки вы хотите развивать?
3. Каковы ваши карьерные цели после окончания учебы?
4. Вам больше интересна техническая сторона или управление проектами и продуктами?

Пожалуйста, поделитесь этой информацией, и я смогу дать более точный совет.


In [53]:
from llama_index.core.workflow import Context
ctx = Context(agent)

In [54]:
response = await agent.run(
    "Привет, я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере",
    ctx=ctx
)
print(response)

2026-02-03 08:44:20,977 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Привет! Это здорово, что у тебя есть опыт в Java и Python, а также в создании ML-сервисов. Учитывая твои интересы и опыт, давай разберемся, какая из магистерских программ - Искусственный интеллект или Управление ИИ-продуктами - может подойти тебе лучше. 

Ты можешь рассказать, какие аспекты программирования и работы с ИИ тебя интересуют больше? Например, ты хочешь углубиться в технические детали разработки ИИ-алгоритмов или больше интересуешься управлением проектами и продуктами в этой области?


In [55]:
response = await agent.run(
    "Подскажи какую программу, мне лучше выбрать исходя из прошлого опыта и то, что мне нравится",
    ctx=ctx
)
print(response)

2026-02-03 08:44:24,344 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:44:24,729 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:44:24,855 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:44:26,648 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:44:26,727 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:44:27,581 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Вот краткое описание обеих программ:

1. **Искусственный интеллект**:
   - Программа охватывает различные области, такие как data science, машинное обучение, глубокое обучение и обработка естественного языка.
   - Студенты работают над реальными проектами с ведущими компаниями, что дает возможность получить практический опыт и наладить контакты.
   - Выпускники могут стать инженерами по машинному обучению, инженерами данных, разработчиками ИИ-продуктов или аналитиками данных.
   - Учебный план включает технические курсы, мастер-классы и развитие мягких навыков.

2. **Управление ИИ-продуктами**:
   - Программа фокусируется на управлении продуктами ИИ, предоставляя глубокие технические знания в разработке систем искусственного интеллекта и навыки управления продуктами.
   - Выпускники могут занять должности менеджеров по продуктам ИИ, менеджеров проектов ИИ, аналитиков данных продуктов и лидеров продуктов ИИ.
   - Учебный план включает мастер-классы, онлайн-лекции и финальный проект с па

In [56]:
response = await agent.run(
     "Хочу уметь создавать микросервисы. Подскажи на какую программу пойти лучше изходя из учебного плана",
    ctx=ctx
)
print(response)

2026-02-03 08:44:34,712 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:44:36,286 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:44:36,289 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 08:44:37,638 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:44:39,226 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-03 08:44:39,818 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Вот информация о учебных планах обеих программ:

1. **Искусственный интеллект**:
   - Включает модули по разработке продуктов данных, Python, глубокому обучению, машинному обучению, статистике и графическим интерфейсам.
   - Охватывает широкий спектр тем, связанных с искусственным интеллектом и его приложениями, но не акцентирует внимание на микросервисах.

2. **Управление ИИ-продуктами**:
   - Включает обязательные дисциплины, такие как разработка продуктов данных, проектирование микросервисов, веб-разработка, программирование на Python, а также управление проектами и продуктами в области ИИ.
   - Также охватывает темы, связанные с бизнес-анализом, стратегическим управлением продуктами и юридическими аспектами разработки ИИ.

Если ты хочешь уметь создавать микросервисы, программа "Управление ИИ-продуктами" будет более подходящей, так как она включает конкретные курсы по проектированию микросервисов и веб-разработке. 

Как ты считаешь, подходит ли тебе эта программа?


In [57]:
response = await agent.run(
     "Разве нет таких же программ и на Искусственном интеллекте?",
    ctx=ctx
)
print(response)

2026-02-03 08:44:44,323 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


В программе "Искусственный интеллект" действительно есть курсы, которые могут быть связаны с разработкой и применением технологий, но акцент на микросервисах и веб-разработке не так выражен, как в программе "Управление ИИ-продуктами". 

Программа "Искусственный интеллект" больше ориентирована на технические аспекты, такие как машинное обучение и глубокое обучение, а также на разработку продуктов данных. Однако, если в ней есть курсы по разработке графических интерфейсов и Python, это может быть полезно для создания микросервисов, но не так явно, как в программе "Управление ИИ-продуктами", где есть конкретные дисциплины по проектированию микросервисов и веб-приложений.

Если создание микросервисов является твоей основной целью, программа "Управление ИИ-продуктами" будет более целенаправленной. Если же ты хочешь углубиться в технические аспекты ИИ и при этом изучить разработку, то "Искусственный интеллект" также может быть хорошим выбором. 

Как ты думаешь, что для тебя важнее: техническ

In [58]:
response = await agent.run(
     "Кем можно работать после обучения?",
    ctx=ctx
)
print(response)

2026-02-03 08:44:48,788 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Вот кем можно работать после обучения по каждой из программ:

1. **Искусственный интеллект**:
   - Инженер по машинному обучению
   - Инженер данных
   - Разработчик ИИ-продуктов
   - Аналитик данных
   - Исследователь в области ИИ
   - Специалист по обработке естественного языка
   - Разработчик алгоритмов и моделей ИИ

2. **Управление ИИ-продуктами**:
   - Менеджер по продуктам ИИ
   - Менеджер проектов ИИ
   - Аналитик данных продуктов
   - Лидер продуктов ИИ
   - Специалист по стратегическому управлению продуктами
   - Бизнес-аналитик в области ИИ
   - Специалист по маркетингу ИИ-продуктов

В зависимости от твоих интересов и выбранной программы, ты сможешь выбрать карьерный путь, который больше всего соответствует твоим целям. Если ты хочешь сосредоточиться на технической стороне и разработке, программа "Искусственный интеллект" может быть более подходящей. Если же тебя интересует управление проектами и продуктами, то "Управление ИИ-продуктами" будет лучшим выбором. 

Какой из этих

In [59]:
index.storage_context.persist(persist_dir="../storage")